### 1. Importing Libraries

In [1]:
import numpy as np, pandas as pd

from sklearn.model_selection import train_test_split

import os

import warnings
warnings.filterwarnings("ignore")

### 2. Reading the data 

In [2]:
PROJECT_DIR = r"/Users/abhisheksaurav/Desktop/flight-sagemaker-project"
DATA_DIR = "data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [4]:
flights = get_data("flight_price")

In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The data set contains 10,684 rows and 11 features. 
- columns 'Route' and 'Total Stops' have a missing values each each
- the data types of some features isn't appropriate. 

### 3. Preleminary Analysis

#### 3.1 Check Data Types

In [6]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/19,Banglore,New Delhi,BLR → DEL,22:20,22/03/24 1:10,2h 50m,non-stop,No info,3897
1,Air India,01/05/19,Kolkata,Banglore,CCU → IXR → BBI → BLR,5:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,09/06/19,Delhi,Cochin,DEL → LKO → BOM → COK,9:25,10/06/24 4:25,19h,2 stops,No info,13882
3,IndiGo,12/05/19,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/19,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [8]:
value = flights.Date_of_Journey.iloc[6]
value

'12/03/19'

In [9]:
value = flights.Dep_Time.iloc[6]
value

'18:55'

In [10]:
value = flights.Duration.iloc[6]
value

'15h 30m'

In [11]:
value = flights.Total_Stops.iloc[6]
value

'1 stop'

#### 3.2 Check for Duplicates 

In [12]:
flights.duplicated().sum()

220

In [13]:
(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(["Airline", "Date_of_Journey", "Source", "Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/19,Banglore,New Delhi,BLR → BOM → AMD → DEL,8:50,02/03/24 23:55,39h 5m,2 stops,No info,17135
9848,Air India,01/03/19,Banglore,New Delhi,BLR → BOM → AMD → DEL,8:50,02/03/24 23:55,39h 5m,2 stops,No info,17135
1495,Air India,01/04/19,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,02/04/24 1:20,15h 20m,2 stops,No info,10408
9913,Air India,01/04/19,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,02/04/24 1:20,15h 20m,2 stops,No info,10408
3598,Air India,01/05/19,Kolkata,Banglore,CCU → GAU → DEL → BLR,9:50,02/05/24 8:55,23h 5m,2 stops,No info,13227
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/19,Banglore,New Delhi,BLR → DEL,5:45,8:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/19,Banglore,New Delhi,BLR → DEL,5:45,8:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/19,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/19,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


#### 3.3 Observation

- The type of `Date of Journey`, `Dep_Time` and `Arrival_Time` should be date time.
- The type of `Duration` and `Total_Stops` is mixed. It should be numeric type. 
- There are `220` duplicates. These should be removed. 

###  4. Detailed Analysis

#### 4.1 Airline

In [14]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [15]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some of the entries has inconsistent/inaccurate values values

In [16]:
(
    flights
    .Airline
    
    .str.replace("Premium economy", "")
    
    .str.replace("Business", "")
    
    .str.title()
    
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

#### 4.2 Date of Journey

In [17]:
flights.Date_of_Journey

0        24/03/19
1        01/05/19
2        09/06/19
3        12/05/19
4        01/03/19
           ...   
10678    09/04/19
10679    27/04/19
10680    27/04/19
10681    01/03/19
10682    09/05/19
Name: Date_of_Journey, Length: 10683, dtype: object

In [18]:
pd.to_datetime(flights.Date_of_Journey)

0       2019-03-24
1       2019-01-05
2       2019-09-06
3       2019-12-05
4       2019-01-03
           ...    
10678   2019-09-04
10679   2019-04-27
10680   2019-04-27
10681   2019-01-03
10682   2019-09-05
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

#### 4.3 Source

In [19]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

#### 4.4 Destination

In [20]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

#### 4.5 Dep_Time

In [21]:
flights.Dep_Time

0        22:20
1         5:50
2         9:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680     8:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [22]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]

)

Series([], Name: Dep_Time, dtype: object)

#### 4.6 Arrival_Time

In [23]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n= 1)
    .str.get(1)
    .unique()
)

array(['1:10', '4:25', '10:25', '5:05', '9:20', '19:15', '12:35', '13:20',
       '8:15', '3:35', '19:00', '11:05', '10:55', '0:15', '2:45', '12:00',
       '1:30', '16:45', '9:25', '4:40', '11:25', '14:25', '14:35', '9:05',
       '1:35', '10:05', '11:10', '7:45', '0:55', '23:35', '0:40', '18:50',
       '7:55', '18:15', '13:15', '9:45', '6:50', '15:15', '7:40', '2:20',
       '18:30', '19:45', '19:50', '1:00', '0:05', '16:20', '0:30', '0:50',
       '13:45', '9:15', '7:10', '8:55', '4:45', '5:35', '9:00', '10:45',
       '22:10', '22:40', '5:25', '2:15', '20:20', '6:35', '1:20', '19:10',
       '10:50', '23:25', '12:15', '10:40', '23:15', '22:55', '9:10',
       '18:10', '0:25', '20:45', '21:05', '23:00', '0:45', '4:35',
       '18:00', '12:30', '2:10', '0:20', '23:20', '12:25', '22:35',
       '20:25', '0:35', '16:50', '9:30', '12:05', '10:35', '15:05',
       '18:40', '9:35', '8:50', '16:40', '19:40', '16:10', '21:20',
       '7:25', '20:40', '13:00', '11:20', '0:10', '8:10', '16:1

#### 4.7 Duration

In [24]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [25]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    #.unique()
)

6474    5m
Name: Duration, dtype: object

In [26]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,06/03/19,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The observation index 6474 has duration 5mins. This is clearly so we will delete this observation. 

In [27]:
(
    flights
    .Duration
    .drop(index = 6474)
    .str.split(" ", expand = True)
    .set_axis(["hour", "minute"], axis = 1)
    .assign(
        hour = lambda df_:(
            df_
            .hour
            .str.replace("h", "")
            .astype("int")
            .mul(60)
            
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype("int")
            
        )
    
    )
    .sum(axis = 1)
    
    
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [28]:
(
    flights
    .Duration
    .drop(index = 6474)
    .str.split(" ", expand = True)
    .set_axis(["hour", "minute"], axis = 1)
    .assign(
        hour = lambda df_:(
            df_
            .hour
            .str.replace("h", "")
            .astype("int")
            .mul(60)
            
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype("int")
            
        )
    
    )
    .sum(axis = 1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index = 6474))
    
    
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


#### 4.8 Total_Stops

In [29]:
(
    flights
    .Total_Stops
    .unique()

)

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [30]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?", "", regex = True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

#### 4.9 Additional_Info

In [31]:
(
    flights
    .Additional_Info
    .unique()
)

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

### 5. Cleaning Operations

In [32]:
def convert_to_mins(ser):
    return (
    ser
    .str.split(" ", expand = True)
    .set_axis(["hour", "minute"], axis = 1)
    .assign(
        hour = lambda df_:(
            df_
            .hour
            .str.replace("h", "")
            .astype("int")
            .mul(60)
            
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype("int")
            
        )
    
    )
    .sum(axis = 1)
    
    
)

In [33]:
def clean_data(df):
    return (
        df
        # .dropna()
        
        .drop(index = [6474])
        
        .drop_duplicates()
        
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include = "O").columns
        })
        
        .rename(columns = str.lower)
        
        .assign(
            airline = lambda df_:(
            df_
                .airline
                .str.replace("Premium economy", "")
                .str.replace("Business", "")
                .str.title()
            ),
            date_of_journey = lambda df_: pd.to_datetime(df_.date_of_journey),
            dep_time = lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_:pd.to_datetime(df_.arrival_time).dt.time,                      
            duration = lambda df_: df_.duration.pipe(convert_to_mins),
            total_stops = lambda df_: (
            df_
            .total_stops
            .replace("non-stop", "0")
            .str.replace(" stops?", "", regex = True)
            .pipe(lambda ser: pd.to_numeric(ser))
            ),
            
            additional_info = lambda df_: df_.additional_info.replace("No info", "No Info")
            
            
        )
        
        .drop(columns = "route")
    )

In [34]:
flights_cleaned  = clean_data(flights)
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-01-05,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-09-06,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-12-05,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-01-03,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-09-04,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-01-03,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


### 6. Split the Data

In [35]:
flights_final = flights_cleaned.copy()

In [36]:
X = flights_final.drop(columns = "price")
y = flights_final.price.copy()

In [37]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size = 0.2, random_state= 42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6695, 9) (6695,)
(1674, 9) (1674,)
(2093, 9) (2093,)


### 7.  Export the Subsets

In [38]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    
    X.join(y).to_csv(file_path, index = False)
    
    return pd.read_csv(file_path).head()

In [39]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-21,Banglore,New Delhi,08:55:00,19:10:00,615,1.0,In-flight meal not included,7832
1,Jet Airways,2019-03-27,Delhi,Cochin,17:30:00,04:25:00,655,1.0,In-flight meal not included,6540
2,Goair,2019-09-03,Banglore,New Delhi,11:40:00,14:35:00,175,0.0,No Info,7305
3,Air India,2019-12-06,Kolkata,Banglore,09:25:00,18:30:00,545,1.0,No Info,8366
4,Jet Airways,2019-12-03,Banglore,New Delhi,22:55:00,07:40:00,525,1.0,In-flight meal not included,11087


In [40]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-03,Banglore,New Delhi,08:00:00,08:15:00,1455,1.0,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0.0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0.0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0.0,No Info,3597
4,Indigo,2019-06-05,Kolkata,Banglore,15:15:00,17:45:00,150,0.0,No Info,4804


In [41]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1.0,No Info,5054
1,Multiple Carriers,2019-12-06,Delhi,Cochin,09:45:00,22:30:00,765,1.0,No Info,9646
2,Jet Airways,2019-12-03,Banglore,New Delhi,22:55:00,15:15:00,980,1.0,In-flight meal not included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1.0,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2.0,No Info,16704
